In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import numpy as np
from matplotlib import image, pyplot as plt
import os, glob, sys, re, random
from google.colab import files

In [0]:
train_dir = "/content/drive/My Drive/DL/DIV2K/train/"
valid_dir = "/content/drive/My Drive/DL/DIV2K/valid/"
dir_type = { 0:"HR/", 1:"LR_bicub_X2/", 2:"LR_bicub_X3/", 3:"LR_bicub_X4/", 
               4:"LR_unkn_X2/", 5:"LR_unkn_X3/", 6:"LR_unkn_X4/" }

In [0]:
#порезать на квадраты

def get_path(batch_size, directory_type, training=True):
    if training == True:
        directory = train_dir
    else: directory = valid_dir
    batch_X = random.sample(glob.glob(os.path.join(directory, directory_type) + "*.png"), batch_size)
    batch_Y = [os.path.join(directory, dir_type[0], re.sub(r"x\d", "", re.split(r"/", Y)[-1])) for Y in batch_X]
    return batch_X, batch_Y
  
def batch_gen(path_X, path_Y, hsv=False):
    for i in range(len(path_X)):
        hr_img, hr_res = open_img(path_Y[i], hr=True, hsv=hsv)
        print(hr_res)
        lr_img, _ = open_img(path_X[i], hr=False, hsv=hsv, res=hr_res)
        yield hr_img, lr_img

def open_img(img_dir, hr=False, hsv=False, res=()):
    img_raw = tf.read_file(img_dir)
    img = tf.image.decode_png(img_raw, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32).eval()
    if hr == False:
        #upscale
        img = tf.image.resize_images(img, res, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        #img = tf.image.resize_images(img, img_pre.shape[0:2], method=tf.image.ResizeMethod.BICUBIC)
    if hsv == True:
        img = tf.image.rgb_to_hsv(img).eval()
    img_res = img.shape[0:2]
    return img, img_res

def save_img(img, filename, hsv=False):
    if hsv == True:
        img = tf.image.hsv_to_rgb(img)
    img = tf.image.convert_image_dtype(img, tf.uint8)
    img_raw = tf.image.encode_png(img).eval()
    return tf.write_file(tf.constant(filename), img_raw)  

In [6]:
sess = tf.Session()
with sess.as_default():
    #if HSV = False => RGB = True
    HSV = True
    BATCH_SIZE = 5
    DIR_TYPE = dir_type[5]
    
    #super-duper counter for save-name
    I = 0
    
    #create paths to batch-images
    lr_path, hr_path = get_path(BATCH_SIZE, DIR_TYPE)
    
    #images generator
    for hr_img, lr_img in batch_gen(lr_path, hr_path, hsv=HSV):
        test_save = save_img(lr_img, "/content/drive/My Drive/DL/test_img/lr{}.png".format(I), hsv=HSV)
        sess.run(test_save)
        I += 1

(1296, 2040)
(1356, 2040)
(1092, 2040)
(1356, 2040)
(1356, 2040)
